In [ ]:

# Read in data about Covid-19 in Chilean districts
cases_districts = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto1/Covid-19.csv')
cases_districts = cases_districts[~cases_districts.Comuna.str.contains('Desconocido')]


deaths_districts = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto38/CasosFallecidosPorComuna.csv')
deaths_districts = deaths_districts[~deaths_districts.Comuna.str.contains('Desconocido')]
deaths_districts = deaths_districts[~deaths_districts.Comuna.str.contains('Total')]

In [ ]:
def timeline_plot(df, title):
    """
    A function to plot a seaborn diagram which shows the development of Covid-19 data in Chilean Regions. 
    For better readability, the function returns a log scale plot. 
    
    Input: 
        df (DataFrame): A DataFrame whith information about Covid-19 in Chile.
        title (string): Specify the kind of the plot (cases or deaths)
        
    Output: 
        A seaborn plot. 
    """

    # Get only relevant columns with information about cases/deaths:     
    df_numbers = df[df.columns[5:-1]]
    df_final = pd.concat([df['Region'], df_numbers], axis = 1)

    # Group by region to get total numbers for each region. 
    df_grouped = df_final.groupby('Region')[[i for i in df_final.columns[1:]]].sum().reset_index()

    # Apply melt, convert Date-column to datetime and sort values
    df_melt = pd.melt(df_grouped, id_vars = 'Region', 
                       value_vars = df_grouped.columns.drop('Region'),
                       var_name='Date', 
                       value_name='Cases')

    df_melt.loc[:, ['Date']] = pd.to_datetime(df_melt['Date'])
    df_melt = df_melt.sort_values('Cases', ascending = False)

    # Plot the timeline
    f, ax = plt.subplots(figsize = (16, 12))

    g = sns.lineplot(
        df_melt.Date, 
        df_melt.Cases, 
        hue = df_melt.Region
    )

    plt.xlabel('Date', fontsize = 20)
    plt.ylabel('{} (log scale)'.format(title), fontsize = 20)
    plt.title('Covid-19 {} in Chile (log scale)'.format(title), fontsize = 25)

    ax.yaxis.tick_right()
    ax.set_yscale('log')
    plt.tick_params(labelsize=20, rotation=90)
    plt.legend(scatterpoints=1, frameon=True, labelspacing=.2, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.5)

    plt.grid(False)
    ax.yaxis.grid()
    sns.despine()
    plt.show()

In [ ]:
    #(np.sum(casen.isna() == True)/casen.shape[0]).sort_values(ascending = False).head(50)

In [ ]:
mobility_chile_target = mobility_chile[mobility_chile['sub_region_1'] == target]

merged_mobility = df_sel.merge(mobility_chile_target, left_on = 'Region', right_on = 'date', how = 'left')
merged_mobility = merged_mobility.drop(['sub_region_1', 'date'], axis = 1)
merged_mobility = merged_mobility[~merged_mobility['retail_and_recreation_percent_change_from_baseline'].isna()]

merged_mobility

In [ ]:
transposed_cases.head(2)

In [ ]:
mean_error = []
for day in range(3, 157):
    train = df_mobility[df_mobility['days'] < day]
    val = df_mobility[df_mobility['days'] == day]

    p = val['yesterday_value'].values

    error = rmsle(val['cases'].values, p)
    print('day %d - Error %.5f' % (day, error))
    mean_error.append(error)
print('Mean Error = %.5f' % np.mean(mean_error))